In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
!pip install geocoder

In [0]:
import geocoder

In [0]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [0]:
soup = BeautifulSoup(data, 'html.parser')

In [0]:
neighborhoodList = []

In [0]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
mb_df = pd.DataFrame({"Neighborhood": neighborhoodList})
mb_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [9]:
print(mb_df.shape)

(42, 1)


# Get coordinates

In [0]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in mb_df["Neighborhood"].tolist() ]

In [12]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00426978951916, 72.85579215153318],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21094000000005, 72.84137000000004],
 [19.048530000000028, 72.93222000000003

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
# merge the coordinates into the original dataframe
mb_df['Latitude'] = df_coords['Latitude']
mb_df['Longitude'] = df_coords['Longitude']

In [15]:
# check the neighborhoods and the coordinates
print(mb_df.shape)
mb_df

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [0]:
# save the DataFrame as CSV file
mb_df.to_csv("mb_df.csv", index=False)

# Create Map of India

In [17]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Geograpical Coordinates of Mumbai, India {}, {}.'.format(latitude, longitude))

The Geograpical Coordinates of Mumbai, India 18.9387711, 72.8353355.


In [18]:
# create map of Toronto using latitude and longitude values
map_mb = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(mb_df['Latitude'], mb_df['Longitude'], mb_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mb)  
    
map_mb

In [0]:
# save the map as HTML file
map_mb.save('map_mb.html')

In [20]:
CLIENT_ID = 'EVKAG3KAWICOL10FDKJYEF5PBTCCR35FSUBMBVHDVIQKAJVD' # your Foursquare ID
CLIENT_SECRET = 'F040HVFLBUKZGO32QYDWLAIKG5F0CHU3ECT5I2EADUWC32J3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EVKAG3KAWICOL10FDKJYEF5PBTCCR35FSUBMBVHDVIQKAJVD
CLIENT_SECRET:F040HVFLBUKZGO32QYDWLAIKG5F0CHU3ECT5I2EADUWC32J3


# Top 100 venues that are within a radius of 5000 meters.

In [0]:
radius = 5000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mb_df['Latitude'], mb_df['Longitude'], mb_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3805, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
2,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
3,Andheri,19.118459,72.841763,Joey's Pizza,19.126762,72.830001,Pizza Place
4,Andheri,19.118459,72.841763,Prithvi Theatre,19.106157,72.825810,Theater


In [24]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,77,77,77,77,77,77
Baiganwadi,100,100,100,100,100,100
Bandra,100,100,100,100,100,100
Bhandup,100,100,100,100,100,100
Borivali,100,100,100,100,100,100
Charkop,100,100,100,100,100,100
Chembur,100,100,100,100,100,100
Dahisar,100,100,100,100,100,100


# Unique categories returned

In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 159 uniques categories.


In [26]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Ice Cream Shop', 'Falafel Restaurant', 'Pizza Place',
       'Theater', 'Pub', 'Breakfast Spot', 'Indian Restaurant',
       'Multiplex', 'Seafood Restaurant', 'Bar', 'Cocktail Bar',
       'American Restaurant', 'Café', 'Food Truck', 'Sandwich Place',
       'Juice Bar', 'Mughlai Restaurant', 'Mediterranean Restaurant',
       'Hotel', 'Brewery', 'Snack Place', 'Maharashtrian Restaurant',
       'Club House', 'Coffee Shop', 'Lounge', 'Dessert Shop',
       'Chinese Restaurant', 'Comfort Food Restaurant',
       'Italian Restaurant', 'Spanish Restaurant', 'Beach', 'Spa',
       'South Indian Restaurant', 'Diner', 'Nightclub', 'BBQ Joint',
       'Restaurant', 'Donut Shop', 'Gym', 'Cupcake Shop',
       'Bengali Restaurant', 'Gym / Fitness Center',
       'Australian Restaurant', 'Food', 'Punjabi Restaurant',
       'Asian Restaurant', 'Garden', 'Golf Course', 'Sculpture Garden'],
      dtype=object)

In [27]:
# check if the results contain "Hotel"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [28]:
# one hot encoding
mb_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mb_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mb_onehot.columns[-1]] + list(mb_onehot.columns[:-1])
mb_onehot = mb_onehot[fixed_columns]

print(mb_onehot.shape)
mb_onehot.head()

(3805, 160)


,Neighborhoods,Afghan Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


# Grouping together

In [29]:
mb_grouped = mb_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mb_grouped.shape)
mb_grouped

(42, 160)


,Neighborhoods,Afghan Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Women's Store
0,Andheri,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.01,0.010000,0.00,0.030000,0.030000,0.03,0.00,0.000000,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.000000,0.00,0.000000,0.00,0.000000,0.060000,0.030000,0.000000,0.01,0.02,0.040000,0.00,0.00,0.00,0.01,0.000000,0.000000,0.00,0.00,0.01,0.00,0.00,0.000000,0.020000,0.00,0.00,0.020000,0.01,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.060000,0.000000,0.030000,0.120000,0.00,0.00,0.010000,0.00,0.00,0.020000,0.040000,0.01,0.000000,0.01,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.00,0.030000,0.00,0.000000,0.00,0.010000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.00,0.000000,0.05,0.000000,0.000000,0.00,0.020000,0.00,0.00,0.020000,0.00,0.000000,0.040000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.01,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
1,Anushakti Nagar,0.00,0.00,0.000000,0.000000,0.00,0.025974,0.00,0.00,0.000000,0.00,0.012987,0.025974,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.012987,0.00,0.000000,0.00,0.000000,0.025974,0.038961,0.000000,0.00,0.00,0.025974,0.00,0.00,0.00,0.00,0.012987,0.012987,0.00,0.00,0.00,0.00,0.00,0.012987,0.038961,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.077922,0.00,0.00,0.012987,0.00,0.000000,0.012987,0.00,0.00,0.00,0.00,0.025974,0.00,0.00,0.012987,0.00,0.00,0.00,0.012987,0.00,0.000000,0.012987,0.012987,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.038961,0.155844,0.00,0.00,0.012987

In [30]:
len(mb_grouped[mb_grouped["Hotel"] > 0])

31

# Creating a new DataFrame containing Hotel data only

In [0]:
mb_hotel = mb_grouped[["Neighborhoods","Hotel"]]

In [32]:
mb_hotel.head()

,Neighborhoods,Hotel
0,Andheri,0.06
1,Anushakti Nagar,0.00
2,Baiganwadi,0.00
3,Bandra,0.03
4,Bhandup,0.01


# Clustering into 5 clusters

In [33]:
# set number of clusters
kclusters = 5

mb_clustering = mb_hotel.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mb_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 3, 2, 1, 3, 3, 2, 3, 3], dtype=int32)

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mb_merged = mb_hotel.copy()

# add clustering labels
mb_merged["Cluster Labels"] = kmeans.labels_

In [35]:
mb_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mb_merged.head()

,Neighborhood,Hotel,Cluster Labels
0,Andheri,0.06,4
1,Anushakti Nagar,0.00,3
2,Baiganwadi,0.00,3
3,Bandra,0.03,2
4,Bhandup,0.01,1


In [36]:
# Merging with Longitude and Lattitude
mb_merged = mb_merged.join(mb_df.set_index("Neighborhood"), on="Neighborhood")

print(mb_merged.shape)
mb_merged.head() # check the last columns!

(42, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,Andheri,0.06,4,19.118459,72.841763
1,Anushakti Nagar,0.00,3,19.042830,72.927340
2,Baiganwadi,0.00,3,19.062940,72.926630
3,Bandra,0.03,2,19.054370,72.840170
4,Bhandup,0.01,1,19.145560,72.948560


# Sort results by Cluster labels

In [37]:
# sort the results by Cluster Labels
print(mb_merged.shape)
mb_merged.sort_values(["Cluster Labels"], inplace=True)
mb_merged

(42, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
17,Kalyan,0.040000,0,18.953937,72.820367
14,Grant Road,0.050000,0,18.959290,72.831080
24,"Matharpacady, Mumbai",0.050000,0,18.950694,72.827268
22,Mahavir Nagar (Kandivali),0.010000,1,19.210940,72.841370
27,Mulund,0.010000,1,19.171830,72.955650
18,Kandivali,0.010000,1,19.211982,72.837573
20,Kausa,0.010000,1,19.127580,72.825390
34,Thakur village,0.010000,1,19.210200,72.875410
4,Bhandup,0.010000,1,19.145560,72.948560
33,"Sonapur, Bhandup",0.010000,1,19.163940,72.935440


# Visualizing the Clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mb_merged['Latitude'], mb_merged['Longitude'], mb_merged['Neighborhood'], mb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Cluster 1

In [40]:
mb_merged.loc[mb_merged['Cluster Labels'] == 0]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
17,Kalyan,0.04,0,18.953937,72.820367
14,Grant Road,0.05,0,18.959290,72.831080
24,"Matharpacady, Mumbai",0.05,0,18.950694,72.827268


# Cluster 2

In [41]:
mb_merged.loc[mb_merged['Cluster Labels'] == 1]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
22,Mahavir Nagar (Kandivali),0.01,1,19.210940,72.841370
27,Mulund,0.01,1,19.171830,72.955650
18,Kandivali,0.01,1,19.211982,72.837573
20,Kausa,0.01,1,19.127580,72.825390
34,Thakur village,0.01,1,19.210200,72.875410
4,Bhandup,0.01,1,19.145560,72.948560
33,"Sonapur, Bhandup",0.01,1,19.163940,72.935440


# Cluster 3

In [42]:
mb_merged.loc[mb_merged['Cluster Labels'] == 2]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
32,"Sion, Mumbai",0.03,2,19.043410,72.863320
29,Pestom sagar,0.03,2,19.070640,72.902170
35,Tilak Nagar (Mumbai),0.03,2,18.996160,72.852790
38,Vikhroli,0.02,2,19.111090,72.927810
40,Western Suburbs (Mumbai),0.03,2,19.197010,72.827680
39,Wadala,0.03,2,19.017200,72.858170
19,Kanjurmarg,0.03,2,19.131380,72.935680
30,Seven Bungalows,0.02,2,19.131342,72.816342
41,Worli,0.03,2,19.007440,72.816880
3,Bandra,0.03,2,19.054370,72.840170


# Cluster 4

In [43]:
mb_merged.loc[mb_merged['Cluster Labels'] == 3]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
6,Charkop,0.0,3,19.208660,72.826120
9,Devipada,0.0,3,19.224690,72.866050
28,Mumbra,0.0,3,19.167632,73.021408
8,Dahisar,0.0,3,19.250030,72.859070
26,Mogra Village,0.0,3,24.375900,75.954570
25,Mira Road,0.0,3,19.265674,72.870681
23,Mankhurd,0.0,3,19.048530,72.932220
2,Baiganwadi,0.0,3,19.062940,72.926630
1,Anushakti Nagar,0.0,3,19.042830,72.927340
5,Borivali,0.0,3,19.229360,72.857510


# Cluster 5

In [44]:
mb_merged.loc[mb_merged['Cluster Labels'] == 4]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
37,Vashi,0.070000,4,19.084650,72.904810
15,Jogeshwari,0.060000,4,19.137920,72.849410
31,Shil Phata,0.074074,4,19.146580,73.040050
12,Ghatkopar,0.060000,4,19.086523,72.909008
21,Kurla,0.060000,4,19.064980,72.880690
36,Uttan,0.064516,4,26.866340,80.938840
0,Andheri,0.060000,4,19.118459,72.841763


# Observations
### Mostly the hotels are scattered in cluster 3 and 5. Because these areas have higher financial capabilities and nearer to Railway Stations and Airport. We can see that cluster 2 and 4 have almost no hotels in the neighborhood. So that we can see these areas as potential areas for opening new hotels. 